In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import joblib  # Hoặc bạn có thể dùng pickle



In [3]:
cols =[
        "ts", #Time Stamp
        "IP_flags", #IP fragmentation
        "IP_DF", #IP Don’t Fragment
        "TCP_dataofs", #TCP data ofset
        "TCP_SYN", #Sync Flag
        "TCP_ACK", #Acknowledgment Flag
        "sport_class", #Source Port Class (IoTDevID classing)
        "dst_IP_diversity",  #Number of Destination IP (by Source IP)
        "dst_port_diversity",#Number of Destination Port (by Source IP)
        "pck_size_sum_of_EW", #EW packet size total.
        "ts_diff", #he time difference of consecutive packets
        "ts_std_WE", #EW packet time Std.
        "ts_sum_of_EW", #EW packet time total.
        "TCP_window_std_WE", #EW TCP Windows size Std.
        "pck_size_mean_2",  #RW (Rolling windows) packet size mean
        "ts_mean_2", #RW packet time mean - Window size =2
        "ts_std_2", #RW packet time Std.- Window size =2
        "TCP_window_mean_2", #RW TCP Windows size mean - Window size =2
        "TCP_SYN_sum", #Number of TCP Sync Flag
        "TCP_ACK_sum", #EW Acknowledgment Flag
        "TCP_SYN_ratio", #TCP_SYN/TCP_SYN_sum
        "TCP_ACK_SR", #TCP_ACK/sum
        "ts_mean_6",#RW packet time mean - Window size =6
        "ts_std_6",#RW packet time Std.- Window size =6
        "pck_size_mean_9",#	RW (Rolling windows) packet size mean - Window size =9
        "ts_mean_9",#RW packet time mean - Window size =9
        "ts_std_9",#RW packet time Std.- Window size =9
        "TCP_window_mean_9",#RW TCP Windows size mean - Window size =9	
        "TCP_ACK_R",#TCP_ACK_sum/sum
]

In [3]:
df = pd.read_csv("./INPUT/SM/DoS-SYN-6.csv",usecols=cols)
df.to_csv("./INPUT/SM/DoS-SYN-6-cols.csv",index=False)
df.head()

,ts,IP_flags,IP_DF,TCP_dataofs,TCP_SYN,TCP_ACK,sport_class,dst_IP_diversity,dst_port_diversity,pck_size_sum_of_EW,...,TCP_ACK_SR,TCP_ACK_R,pck_size_mean_9,ts_mean_9,ts_std_9,TCP_window_mean_9,pck_size_mean_2,ts_mean_2,ts_std_2,TCP_window_mean_2
0,0.000000,1,0,0,0,0,9,1,1,459.0,...,0.0,0.0,459.0,2.063079,3.932673,0.0,459.0,0.030851,0.043630,0.0
1,0.000000,2,0,0,0,0,9,1,1,479.0,...,0.0,0.0,479.0,3.053064,4.462952,0.0,479.0,0.030869,0.043656,0.0
2,0.061702,1,0,0,0,0,9,1,1,918.0,...,0.0,0.0,459.0,2.063079,3.932673,0.0,459.0,0.030851,0.043630,0.0
3,0.061739,2,0,0,0,0,9,1,1,958.0,...,0.0,0.0,479.0,3.053064,4.462952,0.0,479.0,0.030869,0.043656,0.0
4,0.101027,1,0,0,0,0,9,1,1,1377.0,...,0.0,0.0,459.0,2.063079,3.932673,0.0,459.0,0.081365,0.027807,0.0


# Xuất model

In [4]:
# df = pd.read_csv('INPUT/SM/DoS-SYN-1.csv',usecols=cols)#,header=None )
# df=df[cols]
# X_train =df[df.columns[0:-1]]
# y_train=df[df.columns[-1]]

# df = pd.read_csv('INPUT/VAL/VAL-SYN.csv',usecols=cols)#,header=None )
# df=df.fillna(0)
# df=df[cols]
# X_test =df[df.columns[0:-1]]
# y_test=df[df.columns[-1]]

# # X_test = [
# #     [0, 0, 1, 50, 1, 0, 1, 1, 5, 10, 0.1, 0.05, 0.2, 0.1, 50, 0.2, 0.1, 100, 200, 0.9, 0.1, 0.5, 0.1, 10, 0.2, 0.1, 0.05, 0.2, 0.8],  # SYN Flood
# #     [1, 0, 0, 10, 0, 0, 0, 1, 1, 2, 0.05, 0.02, 0.1, 0.01, 5, 0.01, 0.02, 20, 10, 0.2, 0.05, 0.3, 0.01, 5, 0.03, 0.02, 0.01, 0.1, 0.2],  # Normal traffic
# # ]
# # y_test=[1,0]

# model = RandomForestClassifier(bootstrap=False,criterion="gini",max_depth=7,max_features=8,min_samples_split=2,n_estimators=98)
# model.fit(X_train, y_train)
# # Dự đoán trên tập test
# y_pred = model.predict(X_test)
# print(y_pred)
# # Đánh giá hiệu suất
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

# # Xuất mô hình ra file bằng joblib
# joblib.dump(model, "./model/syn_flood_detection.pkl")
# print("Model has been saved to 'svc_model.pkl'")


# Thử nghiệm model

In [5]:
from sklearn.metrics import f1_score

In [4]:
df = pd.read_csv('./test/captured_20241210_202526_1_SW_1.csv',usecols=cols)#,header=None )
df.to_csv('./test/real_a.csv',index=False)

In [16]:
# Tải mô hình từ file
# loaded_model = joblib.load("./model/syn_flood_detection.pkl")
# loaded_model = joblib.load("./model/RF_SYN_1_model.pkl")
loaded_model = joblib.load("./model/NB_SYN_1_model.pkl")

df = pd.read_csv('./test/file_2_model_nhận_diện_sai.csv',usecols=cols)#,header=None )
df=df.fillna(0)
df=df[cols]
# X_test =df[df.columns[0:-1]]
X_test=df
y_test=df[df.columns[-1]]
# Dự đoán trên dữ liệu mới (giả sử bạn có X_new)
new_predictions = loaded_model.predict(X_test)
for i in range(len(new_predictions)):
    if new_predictions[i]==1:
        print("Syn Flood")
print('No of Bengin:',np.count_nonzero(new_predictions == 0))
print('No of Syn Flood:',np.count_nonzero(new_predictions == 1))


Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
Syn Flood
No of Bengin: 143
No of Syn Flood: 57


c:\Users\NHAT\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GaussianNB from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\NHAT\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
